In [1]:
push!(LOAD_PATH,".")

5-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/skh/MultiAgentSensing"
 "."

In [2]:
using Revise
using POMDPs
using POMDPSimulators

In [3]:
using MDMA

In [520]:
sensor = ViewConeSensor(pi/2, 3);
targets = Vector{Target}(undef, 0);
push!(targets, Target(25,25,0));
push!(targets, Target(26,25,0));
push!(targets, Target(26,26,0));
push!(targets, Target(20,15,0));

# push!(targets, Target(3,25,0));
# push!(targets, Target(5,25,0));
# push!(targets, Target(6,26,0));
# push!(targets, Target(4,24,0));
# push!(targets, Target(5,23,0));
grid = Grid(30,30);
horizon = 30
move_dist = 2
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, targets, move_dist);

In [521]:
policy = MDMA.solve_single_robot(model, UAVState(1,1,:N))


[Iteration 1   ] residual:        120 | iteration runtime:    859.474 ms, (     0.859 s total)
[Iteration 2   ] residual:        120 | iteration runtime:    678.346 ms, (      1.54 s total)
[Iteration 3   ] residual:        120 | iteration runtime:    688.868 ms, (      2.23 s total)
[Iteration 4   ] residual:        120 | iteration runtime:    667.894 ms, (      2.89 s total)
[Iteration 5   ] residual:        120 | iteration runtime:    709.276 ms, (       3.6 s total)
UAVState(1, 3, :NE, false)nothing

ValueIterationPolicy:
 MDPState(UAVState(1, 1, :E, false), 0, nothing) -> UAVState(1, 3, :SE, false)
 MDPState(UAVState(1, 2, :E, false), 0, nothing) -> UAVState(1, 4, :SE, false)
 MDPState(UAVState(1, 3, :E, false), 0, nothing) -> UAVState(1, 4, :SE, false)
 MDPState(UAVState(1, 4, :E, false), 0, nothing) -> UAVState(1, 4, :SE, false)
 MDPState(UAVState(1, 5, :E, false), 0, nothing) -> UAVState(1, 7, :SE, false)
 MDPState(UAVState(1, 6, :E, false), 0, nothing) -> UAVState(1, 7, :SE, false)
 MDPState(UAVState(1, 7, :E, false), 0, nothing) -> UAVState(1, 7, :SE, false)
 MDPState(UAVState(1, 8, :E, false), 0, nothing) -> UAVState(1, 7, :SE, false)
 MDPState(UAVState(1, 9, :E, false), 0, nothing) -> UAVState(1, 7, :SE, false)
 MDPState(UAVState(1, 10, :E, false), 0, nothing) -> UAVState(1, 12, :SE, false)
 MDPState(UAVState(1, 11, :E, false), 0, nothing) -> UAVState(1, 13, :SE, false)
 MDPState(UAVState(1, 12, :E, false), 0, nothing) -> UAVState(1, 13, :SE, false)
 MDPState(UAVState(1, 13

In [522]:
function rew_check(model, state, policy)
    prev = MDPState(state)
    next = action(policy, prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end
function check(model, prev, next)
    prev = MDPState(prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end


check (generic function with 1 method)

In [523]:
test_state = UAVState(3,3,:S)
rew_check(model, test_state, policy)

UAVState(3, 3, :S, false) 0
UAVState(3, 5, :SW, false) 0


In [524]:
check(model, test_state, UAVState(3,3,:E))

UAVState(3, 3, :S, false) 0
UAVState(3, 3, :E, false) 0


In [534]:
path = Vector{MDPState}(undef, 0)
state = MDPState(UAVState(1,1,:S))
push!(path, state)
for x in 1:model.horizon
    state = action(policy, state);
    state = MDPState(state);
    push!(path, state);
#     println(reward(model, state, state.state))
end

In [535]:
using Cairo

#Pixels per meter
function init_cairo(model,ppm, buf)
    width = ppm*dims(model.grid)[1]+ 2*buf*ppm
    height = ppm*dims(model.grid)[2]+ 2*buf*ppm
    c = CairoRGBSurface(width,height);
    cr = CairoContext(c);
    draw_background(cr, width, height)
    return (c, cr)
end


function draw_background(cr::CairoContext, width, height)
    save(cr);
    set_source_rgb(cr,1,1,1);    # white
    rectangle(cr,0.0,0.0,width,height); # background
    fill(cr);
    restore(cr);
end

function draw_grid(g::Grid, cr::CairoContext, point_size, ppm, buf)
    set_source_rgba(cr, 0, 0, 0, 0.3);
    x,y,z = dims(g)
    print(dims(g))
    for i in 1:x
        for j in 1:y
            arc(cr, i*ppm + buf*ppm, j*ppm+ buf*ppm, point_size, 0, 2*pi);
            fill(cr);
        end
    end    
end

function draw_target(cr::CairoContext, t::Target, size, buf)
    save(cr)
    set_source_rgba(cr, 0, 0.3, 0.5, 1);
    arc(cr, t.x*ppm + buf*ppm, t.y*ppm + buf*ppm, size, 0, 2*pi);
    fill(cr);
    restore(cr)
end

function draw_targets(cr::CairoContext, targs::Vector{Target}, size, buf)
    for t in targs
        draw_target(cr, t, size, buf)
    end
end

function draw_state(cr::CairoContext, state::UAVState, model, ppm, fade, cfade, buf)
    save(cr)
    move_to(cr,state.x*ppm + buf*ppm, state.y*ppm + buf*ppm)
    fov = model.sensor.fov
    radius = model.sensor.cutoff
    draw_arc(cr, radius, state.x, state.y, state.heading, fov, ppm, fade,cfade, buf)
    restore(cr)
end
function draw_arc(cr::CairoContext, radius, x,y, heading,fov,ppm, fade,cfade, buf) 
    ## original example, following here
    xc = x*ppm + buf*ppm;
    yc = y*ppm + buf*ppm;
    radius = radius*ppm;
    angle1 = dirAngle(heading) + (-fov/2);  # angles are specified
    angle2 = dirAngle(heading) + (fov/2);  # in radians

#     set_source_rgba(cr, 0, 0, 0, fade);
    set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade);
    set_line_width(cr, 5.0);
    arc(cr, xc, yc, radius, angle1, angle2);
    fill(cr)
    stroke(cr);

    # draw helping lines
#     set_source_rgba(cr, cfade, 1-cfade, 1-cfade, fade);
    set_line_width(cr, 6.0);

    # Draw center
    arc(cr, xc, yc, 10.0, 0, 2*pi);
    fill(cr);
    
    arc(cr, xc, yc, radius, angle1, angle1);
    line_to(cr, xc, yc);
#     fill(cr)
    
    arc(cr, xc, yc, radius, angle2, angle2);
    line_to(cr, xc, yc);
#     fill(cr)
    close_path(cr);
    set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade/2);
    fill_preserve(cr);
    
    stroke(cr);
end


# draw_state(cr, UAVState(1,1,:S), model, ppm)


## mark picture with current date


draw_arc (generic function with 3 methods)

In [536]:

function draw_path(model, ppm, path, cutoff, buf)
    c, cr = init_cairo(model, ppm, buf)

    draw_grid(model.grid, cr, 5, ppm, buf)


    # state= path[1]
    # move_to(cr,state.state.x*ppm, state.state.y*ppm)
    # draw_state(cr, state.state, model, ppm, 0.4)


    state=path[1]
    move_to(cr,state.state.x*ppm + buf*ppm, state.state.y*ppm + buf*ppm)
    draw_state(cr, state.state, model, ppm, 0.4, 1/cutoff, buf)
    for (i,state) in enumerate(path[2:cutoff])
        draw_state(cr, state.state, model, ppm, 0.4, i/cutoff, buf)         
    end
    
    state=path[1]
    move_to(cr,state.state.x*ppm + buf*ppm, state.state.y*ppm + buf*ppm)
    for (i,state) in enumerate(path[2:cutoff])
        cfade = i/cutoff
        fade=1
        set_source_rgba(cr, (cfade), (1-cfade)*0.5, (1-cfade)*1.1, fade/2);
#         set_source_rgba(cr, 0.9, 0.6, 0.01, 1);
        set_line_width(cr, 13.0);
        move_to(cr,path[i].state.x*ppm+ buf*ppm, path[i].state.y*ppm + buf*ppm)
        line_to(cr, state.state.x*ppm+buf*ppm, state.state.y*ppm + buf*ppm)
        stroke(cr) 
         
    end

    draw_targets(cr, model.targets, 15, buf)
    write_to_png(c,"sample_arc.png");
end

draw_path (generic function with 3 methods)

In [537]:
draw_path(model, 50, path, model.horizon, 4)

(30, 30, 8)

0